In [1]:
import xarray as xr
import xclim as xc
import numpy as np
from clisops.core import subset
from dask.distributed import Client

import requests
from pathlib import Path
from zipfile import ZipFile

from memory_profiler import memory_usage
import time
from datetime import datetime as dt

from xclim.core.calendar import adjust_doy_calendar
import pandas as pd
from math import floor

In [2]:
c = Client(n_workers=1, threads_per_worker=3, memory_limit='3G', dashboard_address=8784)

# Benchmark xr.quantiles and friends

Compare options for computing percentiles when input data has NaNs. The test case will be `percentile_doy`, in various flavors.

In [4]:
# test data 1
basedir = Path('..')
url = 'https://code.dented-planet.net/Indices.zip'
test_files = [basedir / 'temp'/ 'Indices' / 'tasmin.nc', basedir / 'temp' / 'Indices' / 'tasmax.nc']
zipfile = basedir / 'temp' / 'indices.zip'
outdir = basedir / 'temp' / 'out'
outdir.mkdir(exist_ok=True)

if not all([tf.is_file() for tf in test_files]):
    if not zipfile.is_file():
        print('Downloading zip file')
        r = requests.get(url)
        with open(zipfile,'wb') as f:
            f.write(r.content)

    print('Extracting zip')
    with ZipFile(zipfile, 'r') as zipObj:
        zipObj.extractall('../temp')

ds = xr.open_mfdataset((basedir / 'temp' / 'Indices').glob('tasm*.nc'))
ds = ds.assign(tas=xc.atmos.tg(ds=ds)).chunk({'lat': 10})
ds

<xarray.Dataset>
Dimensions:    (bnds: 2, lat: 160, lon: 320, time: 365)
Coordinates:
  * time       (time) datetime64[ns] 2021-01-01T12:00:00 ... 2021-12-31T12:00:00
  * lat        (lat) float64 -89.14 -88.03 -86.91 -85.79 ... 86.91 88.03 89.14
  * lon        (lon) float64 0.0 1.125 2.25 3.375 ... 355.5 356.6 357.8 358.9
    height     float64 2.0
Dimensions without coordinates: bnds
Data variables:
    time_bnds  (time, bnds) datetime64[ns] dask.array<chunksize=(365, 2), meta=np.ndarray>
    lat_bnds   (lat, bnds) float64 dask.array<chunksize=(10, 2), meta=np.ndarray>
    lon_bnds   (lon, bnds) float64 dask.array<chunksize=(320, 2), meta=np.ndarray>
    tasmax     (time, lat, lon) float32 dask.array<chunksize=(365, 10, 320), meta=np.ndarray>
    tasmin     (time, lat, lon) float32 dask.array<chunksize=(365, 10, 320), meta=np.ndarray>
    tas        (time, lat, lon) float32 dask.array<chunksize=(365, 10, 320), meta=np.ndarray>
Attributes:
    Conventions:                        CF-1.7 CMIP-6.2
    activity_id:                        ScenarioMIP
    branch_method:                      standard
    branch_time_in_child:               60265.0
    branch_time_in_parent:              60265.0
    contact:                            cmip6-data@ec-earth.org
    creation_date:                      2020-11-25T14:21:11Z
    data_specs_version:                 01.00.32
    ece2cmor_git_revision:              b6bd9abb97b54153379aac4768e3f68c22f16...
    experiment:                         update of RCP4.5 based on SSP2
    experiment_id:                      ssp245
    external_variables:                 areacella
    forcing_index:                      1
    frequency:                          day
    further_info_url:                   https://furtherinfo.es-doc.org/CMIP6....
    grid:                               T159L62
    grid_label:                         gr
    history:                            2020-11-25T14:21:11Z ; CMOR rewrote d...
    initialization_index:               1
    institution:                        AEMET, Spain; BSC, Spain; CNR-ISAC, I...
    institution_id:                     EC-Earth-Consortium
    latest_applied_cmor_fixer_version:  None
    latest_ece2cmor_version:            v1.5.0
    mip_era:                            CMIP6
    nominal_resolution:                 100 km
    parent_activity_id:                 CMIP
    parent_experiment_id:               historical
    parent_mip_era:                     CMIP6
    parent_source_id:                   EC-Earth3-Veg-LR
    parent_time_units:                  days since 1850-01-01
    parent_variant_label:               r1i1p1f1
    physics_index:                      1
    product:                            model-output
    realization_index:                  1
    realm:                              atmos
    source:                             EC-Earth3-Veg-LR (2019): \naerosol: n...
    source_id:                          EC-Earth3-Veg-LR
    source_type:                        AOGCM
    sub_experiment:                     none
    sub_experiment_id:                  none
    table_id:                           day
    table_info:                         Creation Date:(28 May 2020) MD5:0a6f3...
    title:                              EC-Earth3-Veg-LR output prepared for ...
    tracking_id:                        hdl:21.14100/ac665a7f-648a-4978-9e34-...
    variable_id:                        tasmin
    variant_label:                      r1i1p1f1
    license:                            CMIP6 model data produced by EC-Earth...
    cmor_version:                       3.5.0

In [5]:
test2_files = [basedir / 'temp' / 'Indices' / 'taspavics.nc', basedir / 'temp' / 'Indices' / 'taspavics_na.nc']

if not all([tf.is_file() for tf in test2_files]):
    url = 'http://pavics.ouranos.ca:8083/twitcher/ows/proxy/thredds/dodsC/datasets/simulations/bias_adjusted/cmip5/ouranos/cb-oura-1.0/day_CanESM2_historical+rcp85_r1i1p1_na10kgrid_qm-moving-50bins-detrend_1950-2100.ncml'
    tas = subset.subset_bbox(xr.open_dataset(url, chunks=dict(time=256, lon=16)), lon_bnds=[-90, -60], lat_bnds=[45, 48], start_date='1981',end_date='2010').tasmin
    taswna = tas.where((tas.time.dt.dayofyear != 21))
    
    tas.to_netcdf(test2_files[0])
    taswna.to_netcdf(test2_files[1])

tas = xr.open_dataset(test2_files[0], chunks={'lon': 10}).tasmin
taswna = xr.open_dataset(test2_files[0], chunks={'lon': 10}).tasmin

In [6]:
def atomic_test(ds, func, outpath=outdir):
    outname = outpath / f"{func.__name__}_{int(dt.now().timestamp())}.nc"
    out = func(ds)
    out.to_netcdf(outname)
    return out
    
def perform_test(ds, func, outpath=outdir, N=1):
    res = []
    for i in range(N):
        print(f'Performing test with {func.__name__} ({i + 1} / {N})')
        t0wc = time.perf_counter()
        t0sys = time.process_time()
        mem_usage = memory_usage((atomic_test, (ds, func, outpath)), interval=0.1)
        res.append(
            {'wall_time': time.perf_counter() - t0wc,
             'sys_time': time.process_time() - t0sys,
             'max_mem_use': max(mem_usage)}
        )
    return res

In [7]:
# Tests
def xclim_023(
    arr: xr.DataArray, window: int = 5, per: float = 0.1
) -> xr.DataArray:
    """Percentile value for each day of the year.
    Return the climatological percentile over a moving window around each day of the year.
    Parameters
    ----------
    arr : xr.DataArray
      Input data.
    window : int
      Number of days around each day of the year to include in the calculation.
    per : float
      Percentile between [0,1]
    Returns
    -------
    xr.DataArray
      The percentiles indexed by the day of the year.
    """
    # TODO: Support percentile array, store percentile in coordinates.
    #  This is supported by DataArray.quantile, but not by groupby.reduce.
    rr = arr.rolling(min_periods=1, center=True, time=window).construct("window")

    # Create empty percentile array
    g = rr.groupby("time.dayofyear")

    p = g.reduce(np.nanpercentile, dim=("time", "window"), q=per * 100)

    # The percentile for the 366th day has a sample size of 1/4 of the other days.
    # To have the same sample size, we interpolate the percentile from 1-365 doy range to 1-366
    if p.dayofyear.max() == 366:
        p = adjust_doy_calendar(p.sel(dayofyear=(p.dayofyear < 366)), arr)

    p.attrs.update(arr.attrs.copy())
    return p

In [8]:
def xclim_024(
    arr: xr.DataArray,
    window: int = 5,
    per: float = 0.1,
    fill_ends=True,
) -> xr.DataArray:
    """Percentile value for each day of the year.
    Return the climatological percentile over a moving window around each day of the year.
    Parameters
    ----------
    arr : xr.DataArray
      Input data.
    window : int
      Number of days around each day of the year to include in the calculation.
    per : float
      Percentile between [0,1]
    fill_ends : boolean
      Pad first and last windows of the time-series with mean of the window to avoid nan values
    Returns
    -------
    xr.DataArray
      The percentiles indexed by the day of the year.
    """
    # TODO: Support percentile array, store percentile in coordinates.
    #  This is supported by DataArray.quantile, but not by groupby.reduce.
    rr = arr.rolling(min_periods=1, center=True, time=window).construct("window")

    # rolling automatically adds some nans to the windows on the time-series edges
    # Optionally fill nans in first and last n=floor(window/2) time steps with average window values for these
    if fill_ends:
        nstep = floor(window / 2)
        tmp1 = rr.isel(time=slice(0, nstep))
        tmp1 = tmp1.fillna(tmp1.mean(dim="window"))
        tmp2 = rr.isel(time=slice(len(rr.time) - nstep, None))
        tmp2 = tmp2.fillna(tmp2.mean(dim="window"))
        rr = xr.concat(
            [tmp1, rr.isel(time=slice(nstep, len(rr.time) - nstep)), tmp2], dim="time"
        )

    ind = pd.MultiIndex.from_arrays(
        (rr.time.dt.year.values, rr.time.dt.dayofyear.values),
        names=("year", "dayofyear"),
    )
    rrr = (
        rr.assign_coords(time=ind)
        .unstack("time")
        .stack(stack_dim=("year", "window"))
        .chunk(dict(stack_dim=-1))
    )

    # identify data where a xr.quantile(skipna=False) is ok
    mask = np.isnan(rrr).sum(dim="stack_dim")
    mask = (mask == 0) | (mask == len(rrr.stack_dim))
    p = rrr.quantile(q=per, dim="stack_dim", skipna=False)

    # if necessary use xr.quantile(skipna=True) only for grid cells / dayofyear with nans
    if np.any(~mask):
        p_nans = rrr.where(~mask, drop=True).quantile(
            q=per, dim="stack_dim", skipna=True
        )
        p = p.where(mask, p_nans.broadcast_like(p))


    # The percentile for the 366th day has a sample size of 1/4 of the other days.
    # To have the same sample size, we interpolate the percentile from 1-365 doy range to 1-366
    if p.dayofyear.max() == 366:
        p = adjust_doy_calendar(p.sel(dayofyear=(p.dayofyear < 366)), arr)

    p.attrs.update(arr.attrs.copy())
    return p

In [28]:
# This version replaces all NaNs by the mean along the window
def _calc_perc(arr, p=[50]):
    """Ufunc-like computing a percentile over the last axis of the array.

    Processes cases with invalid values separately, which makes it more efficent than np.nanpercentile for array with only a few invalid points.

    Parameters
    ----------
    arr : np.array
        Percentile is computed over the last axis.
    p : sequence of floats
        Percentile to compute, between 0 and 100. (the default is 50)

    Returns
    -------
    np.array
    """
    nan_count = np.isnan(arr).sum(axis=-1)
    out = np.moveaxis(np.percentile(arr, p, axis=-1), 0, -1)
    nans = (nan_count > 0) & (nan_count < arr.shape[-1])
    if np.any(nans):
        out_mask = np.stack([nans] * len(p), axis=-1)
        # arr1 = arr.reshape(int(arr.size / arr.shape[-1]), arr.shape[-1])
        # only use nanpercentile where we need it (slow performance compared to standard) :
        out[out_mask] = np.moveaxis(
            np.nanpercentile(arr[nans], p, axis=-1), 0, -1
        ).ravel()
    return out

def xclim_dev(
    arr: xr.DataArray,
    window: int = 5,
    per: float = 10,
) -> xr.DataArray:
    """Percentile value for each day of the year.

    Return the climatological percentile over a moving window around each day of the year.
    All NaNs are skipped.

    Parameters
    ----------
    arr : xr.DataArray
      Input data, a daily frequency is required.
    window : int
      Number of days around each day of the year to include in the calculation.
    per : float or sequence of floats
      Percentile between [0, 100]

    Returns
    -------
    xr.DataArray
      The percentiles indexed by the day of the year.
      For calendars with 366 days, percentiles of doys 1-365 are interpolated to the 1-366 range.
    """
    rr = arr.rolling(min_periods=1, center=True, time=window).construct("window")

    ind = pd.MultiIndex.from_arrays(
        (rr.time.dt.year.values, rr.time.dt.dayofyear.values),
        names=("year", "dayofyear"),
    )
    rrr = (
        rr.assign_coords(time=ind)
        .unstack("time")
        .stack(stack_dim=("year", "window"))
        .chunk(dict(stack_dim=-1))
    )

    if np.isscalar(per):
        per = [per]

    p = xr.apply_ufunc(
        _calc_perc,
        rrr,
        input_core_dims=[['stack_dim']],
        output_core_dims=[['percentiles']],
        keep_attrs=True,
        kwargs=dict(p=per),
        dask='parallelized',
        output_dtypes=[rrr.dtype],
        output_sizes={"percentiles": len(per)}
    )
    p = p.assign_coords(
        percentiles=xr.DataArray(per, dims=('percentiles',))
    )

    # The percentile for the 366th day has a sample size of 1/4 of the other days.
    # To have the same sample size, we interpolate the percentile from 1-365 doy range to 1-366
    if p.dayofyear.max() == 366:
        p = adjust_doy_calendar(p.sel(dayofyear=(p.dayofyear < 366)), arr)

    p.attrs.update(arr.attrs.copy())
    return p

# Test laziness

In [31]:
%%time
out = xclim_024(ds.tas)
# nothing should have been executed

CPU times: user 780 ms, sys: 42.2 ms, total: 822 ms
Wall time: 2.67 s


In [30]:
%%time
out = xclim_dev(ds.tas)

CPU times: user 35.5 ms, sys: 9.41 ms, total: 44.9 ms
Wall time: 42 ms


# Tests with Test dataset 1

In [32]:
res = perform_test(ds.tas, xclim_024, N=5)
print('xclim 0.24 - Version of Travis with fill_ends')
print(f"Mean wall time : {np.mean([r['wall_time'] for r in res])}")

Performing test with xclim_024 (1 / 5)
Performing test with xclim_024 (2 / 5)
Performing test with xclim_024 (3 / 5)
Performing test with xclim_024 (4 / 5)
Performing test with xclim_024 (5 / 5)
xclim 0.24 - Version of Travis with fill_ends
Mean wall time : 22.82041727179894


In [33]:
res = perform_test(ds.tas, xclim_dev, N=5)
print('xclim dev - Version of Pascal with apply_ufunc')
print(f"Mean wall time : {np.mean([r['wall_time'] for r in res])}")

Performing test with xclim_dev (1 / 5)
Performing test with xclim_dev (2 / 5)
Performing test with xclim_dev (3 / 5)
Performing test with xclim_dev (4 / 5)
Performing test with xclim_dev (5 / 5)
xclim dev - Version of Pascal replacing ALL NaNs
Mean wall time : 35.72202802119428


In [10]:
res = perform_test(ds.tas, xclim_023, N=1)
print('xclim 0.23 - Previous laggy version.')
print(f"Mean wall time : {np.mean([r['wall_time'] for r in res])}")

Performing test with xclim_023 (1 / 1)


/home/phobos/.conda/envs/xclim-dev/lib/python3.8/site-packages/dask/array/core.py:1354: FutureWarning: The `numpy.nanpercentile` function is not implemented by Dask array. You may want to use the da.map_blocks function or something similar to silence this warning. Your code may stop working in a future release.
  warnings.warn(
/home/phobos/.conda/envs/xclim-dev/lib/python3.8/site-packages/dask/array/core.py:1354: FutureWarning: The `numpy.nanpercentile` function is not implemented by Dask array. You may want to use the da.map_blocks function or something similar to silence this warning. Your code may stop working in a future release.
  warnings.warn(
/home/phobos/.conda/envs/xclim-dev/lib/python3.8/site-packages/dask/array/core.py:1354: FutureWarning: The `numpy.nanpercentile` function is not implemented by Dask array. You may want to use the da.map_blocks function or something similar to silence this warning. Your code may stop working in a future release.
  warnings.warn(
/home/phob

/home/phobos/.conda/envs/xclim-dev/lib/python3.8/site-packages/dask/array/core.py:1354: FutureWarning: The `numpy.nanpercentile` function is not implemented by Dask array. You may want to use the da.map_blocks function or something similar to silence this warning. Your code may stop working in a future release.
  warnings.warn(
/home/phobos/.conda/envs/xclim-dev/lib/python3.8/site-packages/dask/array/core.py:1354: FutureWarning: The `numpy.nanpercentile` function is not implemented by Dask array. You may want to use the da.map_blocks function or something similar to silence this warning. Your code may stop working in a future release.
  warnings.warn(
/home/phobos/.conda/envs/xclim-dev/lib/python3.8/site-packages/dask/array/core.py:1354: FutureWarning: The `numpy.nanpercentile` function is not implemented by Dask array. You may want to use the da.map_blocks function or something similar to silence this warning. Your code may stop working in a future release.
  warnings.warn(
/home/phob

/home/phobos/.conda/envs/xclim-dev/lib/python3.8/site-packages/dask/array/core.py:1354: FutureWarning: The `numpy.nanpercentile` function is not implemented by Dask array. You may want to use the da.map_blocks function or something similar to silence this warning. Your code may stop working in a future release.
  warnings.warn(
/home/phobos/.conda/envs/xclim-dev/lib/python3.8/site-packages/dask/array/core.py:1354: FutureWarning: The `numpy.nanpercentile` function is not implemented by Dask array. You may want to use the da.map_blocks function or something similar to silence this warning. Your code may stop working in a future release.
  warnings.warn(
/home/phobos/.conda/envs/xclim-dev/lib/python3.8/site-packages/dask/array/core.py:1354: FutureWarning: The `numpy.nanpercentile` function is not implemented by Dask array. You may want to use the da.map_blocks function or something similar to silence this warning. Your code may stop working in a future release.
  warnings.warn(
/home/phob

/home/phobos/.conda/envs/xclim-dev/lib/python3.8/site-packages/dask/array/core.py:1354: FutureWarning: The `numpy.nanpercentile` function is not implemented by Dask array. You may want to use the da.map_blocks function or something similar to silence this warning. Your code may stop working in a future release.
  warnings.warn(
/home/phobos/.conda/envs/xclim-dev/lib/python3.8/site-packages/dask/array/core.py:1354: FutureWarning: The `numpy.nanpercentile` function is not implemented by Dask array. You may want to use the da.map_blocks function or something similar to silence this warning. Your code may stop working in a future release.
  warnings.warn(
/home/phobos/.conda/envs/xclim-dev/lib/python3.8/site-packages/dask/array/core.py:1354: FutureWarning: The `numpy.nanpercentile` function is not implemented by Dask array. You may want to use the da.map_blocks function or something similar to silence this warning. Your code may stop working in a future release.
  warnings.warn(
/home/phob

...

/home/phobos/.conda/envs/xclim-dev/lib/python3.8/site-packages/dask/array/core.py:1354: FutureWarning: The `numpy.nanpercentile` function is not implemented by Dask array. You may want to use the da.map_blocks function or something similar to silence this warning. Your code may stop working in a future release.
  warnings.warn(
/home/phobos/.conda/envs/xclim-dev/lib/python3.8/site-packages/dask/array/core.py:1354: FutureWarning: The `numpy.nanpercentile` function is not implemented by Dask array. You may want to use the da.map_blocks function or something similar to silence this warning. Your code may stop working in a future release.
  warnings.warn(
/home/phobos/.conda/envs/xclim-dev/lib/python3.8/site-packages/dask/array/core.py:1354: FutureWarning: The `numpy.nanpercentile` function is not implemented by Dask array. You may want to use the da.map_blocks function or something similar to silence this warning. Your code may stop working in a future release.
  warnings.warn(
/home/phob

/home/phobos/.conda/envs/xclim-dev/lib/python3.8/site-packages/dask/array/core.py:1354: FutureWarning: The `numpy.nanpercentile` function is not implemented by Dask array. You may want to use the da.map_blocks function or something similar to silence this warning. Your code may stop working in a future release.
  warnings.warn(
/home/phobos/.conda/envs/xclim-dev/lib/python3.8/site-packages/dask/array/core.py:1354: FutureWarning: The `numpy.nanpercentile` function is not implemented by Dask array. You may want to use the da.map_blocks function or something similar to silence this warning. Your code may stop working in a future release.
  warnings.warn(
/home/phobos/.conda/envs/xclim-dev/lib/python3.8/site-packages/dask/array/core.py:1354: FutureWarning: The `numpy.nanpercentile` function is not implemented by Dask array. You may want to use the da.map_blocks function or something similar to silence this warning. Your code may stop working in a future release.
  warnings.warn(
/home/phob

/home/phobos/.conda/envs/xclim-dev/lib/python3.8/site-packages/dask/array/core.py:1354: FutureWarning: The `numpy.nanpercentile` function is not implemented by Dask array. You may want to use the da.map_blocks function or something similar to silence this warning. Your code may stop working in a future release.
  warnings.warn(
/home/phobos/.conda/envs/xclim-dev/lib/python3.8/site-packages/dask/array/core.py:1354: FutureWarning: The `numpy.nanpercentile` function is not implemented by Dask array. You may want to use the da.map_blocks function or something similar to silence this warning. Your code may stop working in a future release.
  warnings.warn(
/home/phobos/.conda/envs/xclim-dev/lib/python3.8/site-packages/dask/array/core.py:1354: FutureWarning: The `numpy.nanpercentile` function is not implemented by Dask array. You may want to use the da.map_blocks function or something similar to silence this warning. Your code may stop working in a future release.
  warnings.warn(
/home/phob

/home/phobos/.conda/envs/xclim-dev/lib/python3.8/site-packages/dask/array/core.py:1354: FutureWarning: The `numpy.nanpercentile` function is not implemented by Dask array. You may want to use the da.map_blocks function or something similar to silence this warning. Your code may stop working in a future release.
  warnings.warn(
/home/phobos/.conda/envs/xclim-dev/lib/python3.8/site-packages/dask/array/core.py:1354: FutureWarning: The `numpy.nanpercentile` function is not implemented by Dask array. You may want to use the da.map_blocks function or something similar to silence this warning. Your code may stop working in a future release.
  warnings.warn(
/home/phobos/.conda/envs/xclim-dev/lib/python3.8/site-packages/dask/array/core.py:1354: FutureWarning: The `numpy.nanpercentile` function is not implemented by Dask array. You may want to use the da.map_blocks function or something similar to silence this warning. Your code may stop working in a future release.
  warnings.warn(
/home/phob

xclim 0.23 - Previous laggy version.
Mean wall time : 2198.434556741995


## Verification of xclim_dev

In [25]:
outdev = atomic_test(ds.tas.isel(lat=0, lon=0), xclim_dev)
out24 = atomic_test(ds.tas.isel(lat=0, lon=0), xclim_024)
diff = (out24 - outdev)
(diff == 0).all().values

array(True)

In [26]:
out23 = atomic_test(ds.tas.isel(lat=0, lon=0), xclim_023)
diff = (out24 - out23)
(diff == 0).all().values

/home/phobos/.conda/envs/xclim-dev/lib/python3.8/site-packages/dask/array/core.py:1354: FutureWarning: The `numpy.nanpercentile` function is not implemented by Dask array. You may want to use the da.map_blocks function or something similar to silence this warning. Your code may stop working in a future release.
  warnings.warn(
/home/phobos/.conda/envs/xclim-dev/lib/python3.8/site-packages/dask/array/core.py:1354: FutureWarning: The `numpy.nanpercentile` function is not implemented by Dask array. You may want to use the da.map_blocks function or something similar to silence this warning. Your code may stop working in a future release.
  warnings.warn(
/home/phobos/.conda/envs/xclim-dev/lib/python3.8/site-packages/dask/array/core.py:1354: FutureWarning: The `numpy.nanpercentile` function is not implemented by Dask array. You may want to use the da.map_blocks function or something similar to silence this warning. Your code may stop working in a future release.
  warnings.warn(
/home/phob

/home/phobos/.conda/envs/xclim-dev/lib/python3.8/site-packages/dask/array/core.py:1354: FutureWarning: The `numpy.nanpercentile` function is not implemented by Dask array. You may want to use the da.map_blocks function or something similar to silence this warning. Your code may stop working in a future release.
  warnings.warn(
/home/phobos/.conda/envs/xclim-dev/lib/python3.8/site-packages/dask/array/core.py:1354: FutureWarning: The `numpy.nanpercentile` function is not implemented by Dask array. You may want to use the da.map_blocks function or something similar to silence this warning. Your code may stop working in a future release.
  warnings.warn(
/home/phobos/.conda/envs/xclim-dev/lib/python3.8/site-packages/dask/array/core.py:1354: FutureWarning: The `numpy.nanpercentile` function is not implemented by Dask array. You may want to use the da.map_blocks function or something similar to silence this warning. Your code may stop working in a future release.
  warnings.warn(
/home/phob

/home/phobos/.conda/envs/xclim-dev/lib/python3.8/site-packages/dask/array/core.py:1354: FutureWarning: The `numpy.nanpercentile` function is not implemented by Dask array. You may want to use the da.map_blocks function or something similar to silence this warning. Your code may stop working in a future release.
  warnings.warn(
/home/phobos/.conda/envs/xclim-dev/lib/python3.8/site-packages/dask/array/core.py:1354: FutureWarning: The `numpy.nanpercentile` function is not implemented by Dask array. You may want to use the da.map_blocks function or something similar to silence this warning. Your code may stop working in a future release.
  warnings.warn(
/home/phobos/.conda/envs/xclim-dev/lib/python3.8/site-packages/dask/array/core.py:1354: FutureWarning: The `numpy.nanpercentile` function is not implemented by Dask array. You may want to use the da.map_blocks function or something similar to silence this warning. Your code may stop working in a future release.
  warnings.warn(
/home/phob

/home/phobos/.conda/envs/xclim-dev/lib/python3.8/site-packages/dask/array/core.py:1354: FutureWarning: The `numpy.nanpercentile` function is not implemented by Dask array. You may want to use the da.map_blocks function or something similar to silence this warning. Your code may stop working in a future release.
  warnings.warn(
/home/phobos/.conda/envs/xclim-dev/lib/python3.8/site-packages/dask/array/core.py:1354: FutureWarning: The `numpy.nanpercentile` function is not implemented by Dask array. You may want to use the da.map_blocks function or something similar to silence this warning. Your code may stop working in a future release.
  warnings.warn(
/home/phobos/.conda/envs/xclim-dev/lib/python3.8/site-packages/dask/array/core.py:1354: FutureWarning: The `numpy.nanpercentile` function is not implemented by Dask array. You may want to use the da.map_blocks function or something similar to silence this warning. Your code may stop working in a future release.
  warnings.warn(
/home/phob

/home/phobos/.conda/envs/xclim-dev/lib/python3.8/site-packages/dask/array/core.py:1354: FutureWarning: The `numpy.nanpercentile` function is not implemented by Dask array. You may want to use the da.map_blocks function or something similar to silence this warning. Your code may stop working in a future release.
  warnings.warn(
/home/phobos/.conda/envs/xclim-dev/lib/python3.8/site-packages/dask/array/core.py:1354: FutureWarning: The `numpy.nanpercentile` function is not implemented by Dask array. You may want to use the da.map_blocks function or something similar to silence this warning. Your code may stop working in a future release.
  warnings.warn(
/home/phobos/.conda/envs/xclim-dev/lib/python3.8/site-packages/dask/array/core.py:1354: FutureWarning: The `numpy.nanpercentile` function is not implemented by Dask array. You may want to use the da.map_blocks function or something similar to silence this warning. Your code may stop working in a future release.
  warnings.warn(
/home/phob

...

/home/phobos/.conda/envs/xclim-dev/lib/python3.8/site-packages/dask/array/core.py:1354: FutureWarning: The `numpy.nanpercentile` function is not implemented by Dask array. You may want to use the da.map_blocks function or something similar to silence this warning. Your code may stop working in a future release.
  warnings.warn(
/home/phobos/.conda/envs/xclim-dev/lib/python3.8/site-packages/dask/array/core.py:1354: FutureWarning: The `numpy.nanpercentile` function is not implemented by Dask array. You may want to use the da.map_blocks function or something similar to silence this warning. Your code may stop working in a future release.
  warnings.warn(
/home/phobos/.conda/envs/xclim-dev/lib/python3.8/site-packages/dask/array/core.py:1354: FutureWarning: The `numpy.nanpercentile` function is not implemented by Dask array. You may want to use the da.map_blocks function or something similar to silence this warning. Your code may stop working in a future release.
  warnings.warn(
/home/phob

/home/phobos/.conda/envs/xclim-dev/lib/python3.8/site-packages/dask/array/core.py:1354: FutureWarning: The `numpy.nanpercentile` function is not implemented by Dask array. You may want to use the da.map_blocks function or something similar to silence this warning. Your code may stop working in a future release.
  warnings.warn(
/home/phobos/.conda/envs/xclim-dev/lib/python3.8/site-packages/dask/array/core.py:1354: FutureWarning: The `numpy.nanpercentile` function is not implemented by Dask array. You may want to use the da.map_blocks function or something similar to silence this warning. Your code may stop working in a future release.
  warnings.warn(
/home/phobos/.conda/envs/xclim-dev/lib/python3.8/site-packages/dask/array/core.py:1354: FutureWarning: The `numpy.nanpercentile` function is not implemented by Dask array. You may want to use the da.map_blocks function or something similar to silence this warning. Your code may stop working in a future release.
  warnings.warn(
/home/phob

/home/phobos/.conda/envs/xclim-dev/lib/python3.8/site-packages/dask/array/core.py:1354: FutureWarning: The `numpy.nanpercentile` function is not implemented by Dask array. You may want to use the da.map_blocks function or something similar to silence this warning. Your code may stop working in a future release.
  warnings.warn(
/home/phobos/.conda/envs/xclim-dev/lib/python3.8/site-packages/dask/array/core.py:1354: FutureWarning: The `numpy.nanpercentile` function is not implemented by Dask array. You may want to use the da.map_blocks function or something similar to silence this warning. Your code may stop working in a future release.
  warnings.warn(
/home/phobos/.conda/envs/xclim-dev/lib/python3.8/site-packages/dask/array/core.py:1354: FutureWarning: The `numpy.nanpercentile` function is not implemented by Dask array. You may want to use the da.map_blocks function or something similar to silence this warning. Your code may stop working in a future release.
  warnings.warn(
/home/phob

/home/phobos/.conda/envs/xclim-dev/lib/python3.8/site-packages/dask/array/core.py:1354: FutureWarning: The `numpy.nanpercentile` function is not implemented by Dask array. You may want to use the da.map_blocks function or something similar to silence this warning. Your code may stop working in a future release.
  warnings.warn(
/home/phobos/.conda/envs/xclim-dev/lib/python3.8/site-packages/dask/array/core.py:1354: FutureWarning: The `numpy.nanpercentile` function is not implemented by Dask array. You may want to use the da.map_blocks function or something similar to silence this warning. Your code may stop working in a future release.
  warnings.warn(
/home/phobos/.conda/envs/xclim-dev/lib/python3.8/site-packages/dask/array/core.py:1354: FutureWarning: The `numpy.nanpercentile` function is not implemented by Dask array. You may want to use the da.map_blocks function or something similar to silence this warning. Your code may stop working in a future release.
  warnings.warn(
/home/phob

array(False)

# Tests with Test dataset 2

In [10]:
res = perform_test(tas, xclim_024, N=5)
print('xclim 0.24 - Version of Travis with fill_ends')
print(f"Mean wall time : {np.mean([r['wall_time'] for r in res])}")

res = perform_test(taswna, xclim_024, N=5)
print('xclim 0.24 - Version of Travis with fill_ends - with NaN')
print(f"Mean wall time : {np.mean([r['wall_time'] for r in res])}")

Performing test with xclim_024 (1 / 5)
Performing test with xclim_024 (2 / 5)
Performing test with xclim_024 (3 / 5)
Performing test with xclim_024 (4 / 5)
Performing test with xclim_024 (5 / 5)
xclim 0.24 - Version of Travis with fill_ends
Mean wall time : 86.16731039119186
Performing test with xclim_024 (1 / 5)
Performing test with xclim_024 (2 / 5)
Performing test with xclim_024 (3 / 5)
Performing test with xclim_024 (4 / 5)
Performing test with xclim_024 (5 / 5)
xclim 0.24 - Version of Travis with fill_ends - with NaN
Mean wall time : 174.39460554339456


In [12]:
res = perform_test(tas, xclim_dev, N=3)
print('xclim dev - Version of Pascal replacing ALL NaNs')
print(f"Mean wall time : {np.mean([r['wall_time'] for r in res])}")

res = perform_test(taswna, xclim_dev, N=3)
print('xclim dev - Version of Pascal replacing ALL NaNs - with NaN')
print(f"Mean wall time : {np.mean([r['wall_time'] for r in res])}")

Performing test with xclim_dev (1 / 3)
Performing test with xclim_dev (2 / 3)
Performing test with xclim_dev (3 / 3)
xclim dev - Version of Pascal replacing ALL NaNs
Mean wall time : 62.10716049099574
Performing test with xclim_dev (1 / 3)
Performing test with xclim_dev (2 / 3)
Performing test with xclim_dev (3 / 3)
xclim dev - Version of Pascal replacing ALL NaNs - with NaN
Mean wall time : 64.38503600898548


In [ ]:
res = perform_test(tas, xclim_023, N=1)
print('xclim 0.23 - Previous laggy version.')
print(f"Mean wall time : {np.mean([r['wall_time'] for r in res])}")

res = perform_test(taswna, xclim_023, N=1)
print('xclim 0.23 - Previous laggy version. - with NaN')
print(f"Mean wall time : {np.mean([r['wall_time'] for r in res])}")

## Verification of xclim_dev

In [ ]:
outdev = atomic_test(tas, xclim_dev)
out24 = atomic_test(tas, xclim_024)
diff = (out24 - outdev)
(diff == 0).all().values

In [13]:
outdev = atomic_test(taswna, xclim_dev)
out24 = atomic_test(taswna, xclim_024)
diff = (out24 - outdev)
diff.fillna(0) == 0).all().values

array(False)